In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam
import keras
from keras.optimizers.legacy import Adam as LegacyAdam

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pickle as pkl
from src.utils.data_transform import *
from src.utils.data_io import load_prediction_data
import pandas as pd
import os 
import pickle 
import json
from src.utils.data_io import save_data

In [ ]:
results = []

In [ ]:
with open("../data/dataset-info-json/subject_to_indices.json", "r") as f:
    subject_to_indices = json.load(f)

subject_to_indices = {int(k): v for k, v in subject_to_indices.items()}

In [ ]:
# path_to_data = "../data/lstm_training_data/raw/training_data.pkl"
path_to_data = "../data/lstm_training_data/processed/balanced_training_data.pkl"
path_to_models = "../models/lstm"

In [ ]:
with open(path_to_data, "rb") as f:
    data = pickle.load(f)

In [ ]:
for test_subject in subject_to_indices.keys():
    model_path = f"{path_to_models}/EFF_model_{test_subject}.keras"
    model = keras.models.load_model(model_path, compile=False)
    model.compile(optimizer=LegacyAdam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=["accuracy"])
    _, _, test_data, test_labels = load_prediction_data(test_subject, subject_to_indices, data)
    results.append(model.evaluate(test_data, test_labels))

In [ ]:
print(f"Easy Accuracy: {100*np.mean([item[1] for item in results]):.2f}%")
print(f"Loss: {100*np.mean([item[0] for item in results]):.2f}%")

Load training info

In [ ]:
tot_acc = []
tot_res = []
tot_loss = []
info_path = "../models/lstm/training_info/"
for i in range(1,13):
    with open(f"{info_path}/EFFaccuracy_{i}.pkl", "rb") as a:
        tot_acc.append(pickle.load(a))
    with open(f"{info_path}/EFFresults_{i}.pkl", "rb") as r:
        tot_res.append(pickle.load(r))
    with open(f"{info_path}/EFFloss_{i}.pkl", "rb") as l:
        tot_loss.append(pickle.load(l))

tot_res = np.concatenate(tot_res)
tot_acc = np.concatenate(tot_acc)
tot_loss = np.concatenate(tot_loss)

print(f"Evaluation: {np.mean(tot_res[:,1])*100}%")
print(f"Accuracy: {np.mean(tot_acc)*100}%")
print(f"Loss: {np.mean(tot_res[:,0])*100}%")